# HMM Model:

In [1]:
import numpy as np
PI=np.array([0.5,0.5])
A=np.array([[0.999,0.001],[0.2,0.8]])
B=np.array([[0.05,0.35,0.20,0.20,0.20],[0.5,0.1,0.3,0.1,0.0]])

print("PI: ",PI)
print("A: ",A)
print("B: ",B)
obs_map = {0:'Movement'
            , 1:'Passive Social Networking'
            , 2:'Active Social Networking'
            , 3:'Texting'
            , 4:'Access Psych Health App/Sites'}
Observables=np.array([0,3,0,3])

PI:  [ 0.5  0.5]
A:  [[ 0.999  0.001]
 [ 0.2    0.8  ]]
B:  [[ 0.05  0.35  0.2   0.2   0.2 ]
 [ 0.5   0.1   0.3   0.1   0.  ]]


# Filtering:

In [2]:

def forward(PI,A,B,Observables): 
    alpha=np.zeros(shape=(len(PI),len(Observables)))    
    alpha[:,0]=PI*B[:,Observables[0]]
    for i in range(0,len(Observables)-1):      
        alpha[:,i+1]= (np.matmul(alpha[:,i],A))* (B[:,Observables[i+1]])
    return alpha
alpha=forward(PI,A,B,Observables)
print("Alpha unnormalized: ",alpha)

Alpha unnormalized:  [[ 0.025       0.014995    0.00094903  0.00050996]
 [ 0.25        0.0200025   0.0080085   0.00064077]]


In [3]:

def forward_normalized(PI,A,B,Observables):
    c = np.zeros(shape=(len(Observables),))
    alpha = forward(PI,A,B,Observables)
    alpha_normalized = np.zeros(shape=(len(PI),len(Observables)))
    c[0] = 1/(sum(alpha[:,0]))
    alpha_normalized[:,0] = alpha[:,0]*c[0]
    
    # alpha notmalized the way its done in recitation
    for i in range(0,len(alpha[0])-1):
        alpha_normalized[:,i+1]= (np.matmul(alpha_normalized[:,i],A))* (B[:,Observables[i+1]])
        c[i+1] = 1/(sum(alpha_normalized[:,i+1]))
        alpha_normalized[:,i+1]=alpha_normalized[:,i+1]*c[i+1]
    return alpha_normalized,c

a,c = (forward_normalized(PI,A,B,Observables))
print("Alpha normalized:",a)
print("C:",c)
        

Alpha normalized: [[ 0.09090909  0.42845918  0.10594729  0.44315801]
 [ 0.90909091  0.57154082  0.89405271  0.55684199]]
C: [ 3.63636364  7.85770412  3.9070512   7.78420997]


# Evaluation:

In [4]:

def evaluation_unnormalized(alpha):
    return(np.sum(alpha))
print("Evaluation un-normalized: ",evaluation_unnormalized(alpha))

Evaluation un-normalized:  0.320105752597


In [5]:
def evaluation_normalized(c):
    total = 0.0
    for i in range(len(c)):
        total = total + np.log(c[i]) # used natural log to compute the log probablity
    return (-total)
print("Evaluation normalized: ",evaluation_normalized(c))
    

Evaluation normalized:  -6.76735888793


# Backward Probability:

In [6]:

def backward(PI,A,B,Observables): 
    beta=np.zeros(shape=(len(PI),len(Observables)))
    beta[:,len(Observables)-1]=1 
    for j in range(len(Observables)-2,-1,-1):
        beta[:,j]=(np.matmul(A,B[:,Observables[j+1]]))* beta[:,j+1]
    return beta
beta=backward(PI,A,B,Observables)
print("Beta un-normalized: ",beta)

Beta un-normalized:  [[ 0.00201598  0.01008495  0.1999      1.        ]
 [ 0.005904    0.0492      0.12        1.        ]]


In [7]:
def backward_normalized(PI,A,B,c,Observables):
    beta=backward(PI,A,B,Observables)
    beta_normalized = np.zeros(shape=(len(PI),len(Observables)))
    for i in range(0,len(Observables)):
        beta_normalized[:,i] = beta[:,i]*c[i]
    return beta_normalized
print("Beta normalized: ",backward_normalized(PI,A,B,c,Observables))      

Beta normalized:  [[  7.33084547e-03   7.92445925e-02   7.81019535e-01   7.78420997e+00]
 [  2.14690909e-02   3.86599043e-01   4.68846144e-01   7.78420997e+00]]


# Smoothing by Forward-Backward Algorithm:

In [8]:
def smoothed_probability(alpha,beta,noStates, noOfTimeSteps):
    smoothed_matrix=np.zeros(shape=(noStates,noOfTimeSteps))
    for j in range(noOfTimeSteps):
        smoothed_matrix[:,j]=(alpha[:,j]*beta[:,j])/np.dot(alpha[:,j],beta[:,j])        
    return smoothed_matrix

print("Smoothed Probability matrix: ",smoothed_probability(alpha,beta,len(PI), len(Observables)))

Smoothed Probability matrix:  [[ 0.03301859  0.13319621  0.16486072  0.44315801]
 [ 0.96698141  0.86680379  0.83513928  0.55684199]]
